In [ ]:
import sys, os
import numpy as np
from math import *
import math
from PIL import Image
from matplotlib import pyplot
from matplotlib.patches import Rectangle, Circle
import matplotlib.pyplot as plt
import cv2
import random
import dlib

In [ ]:
def bb_intersection_over_union(box1, box2):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(box1[0], box2[0])
	yA = max(box1[1], box2[1])
	xB = min(box1[2], box2[2])
	yB = min(box1[3], box2[3])
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	box1Area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
	box2Area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)
	iou = interArea / float(box1Area + box2Area - interArea)
	return iou

In [ ]:
def calculate_iou(gtbox,HOGbox):
  no_counter = 0
  text = img_file + "\n" + str(num_faces) + "," + str(len(HOGbox)) + "\n"
  for boxA in gtbox:
    if len(HOGbox)==0:
      text += ','.join([str(elem) for elem in boxA]) + '/' + "None" + '/' + "0.0" + '/' + "0.0" + '\n'
      no_counter += 1
    else:
      test_iou = []
      for boxB in HOGbox:
        #print(boxB[0])
        box_iou = (bb_intersection_over_union(boxA, boxB[0]))
        text += ','.join([str(elem) for elem in boxA]) + "/" + ','.join([str(elem) for elem in boxB[0]]) + "/" + str(boxB[1]) + "/" + str(box_iou) + "\n"

      """#FN
      if max(test_iou) == 0.0:
        text += ','.join([str(elem) for elem in boxA]) + "/" + "None" + '/' + "0.0" + "/" + str(max(test_iou)) + "\n"
        no_counter += 1
      #TP
      else:
        for i,u in enumerate(test_iou):
          if u==max(test_iou):
            index=i
            text += ','.join([str(elem) for elem in boxA]) + "/" + ','.join([str(elem) for elem in HOGbox[i][0]]) + "/" + str(HOGbox[i][1]) + "/" + str(max(test_iou)) + "\n"
            break
            """
    """#FP
    for boxC in HOGbox:
      test_iou=[]
      for boxD in gtbox:
        test_iou.append(bb_intersection_over_union(boxC, boxD))
        if max(test_iou)==0.0:
          text += 'None' + '/' + ','.join([str(elem) for elem in boxC])' + '/' + str(correct_iou) + '\n' """
  
  print(text)
  return text

In [ ]:
folds = ["01","02","03","04","05","06","07","08","09","10"]
for fold_num in folds:

  rect_filename = '/content/drive/MyDrive/paper_face_recognitioin/FDDB-folds/rect/FDDB-fold-{f_name}-rectList.txt'.format(f_name = fold_num)
  iou_filename = '/content/drive/MyDrive/paper_face_recognitioin/FDDB-folds/iou/HOG/FDDB-fold-{f_name}-HOG-IOU.txt'.format(f_name = fold_num)

  with open(rect_filename) as f:
    lines = [line.rstrip('\n') for line in f]

  counter = 0
  #f = open(iou_filename,'w')
  while counter < len(lines):
    img_file = lines[counter] 
    image = cv2.imread(img_file)
    num_faces = int(lines[counter+1])
    gtbox = []
    for j in range(num_faces):
      gt = [int(s) for s in lines[counter+j+2].split(',')]
      gtbox.append(gt)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    detector = dlib.get_frontal_face_detector()
    faces, scores, idx = detector.run(image, 1, -1)
    HOGrect = []
    Confidence = []
    #print(faces)
    for i, s in enumerate(scores):
      #n = (s-min_range) / (max_range-min_range)
      Confidence.append(s)
    for face in faces:
      x1 = face.left() 
      y1 = face.top()
      x2 = face.right()
      y2 = face.bottom()
      HOG = [x1,y1,x2,y2]
      HOGrect.append(HOG)
    HOGbox = list(zip(HOGrect,Confidence))
    #print(HOGbox)

    text = calculate_iou(gtbox,HOGbox)
    
    #f.write(text)
    counter = counter + num_faces + 2

  try:
    print(no_counter)
  except: 
    pass
  print(fold_num,"is done")
  f.close()

/content/drive/MyDrive/paper_face_recognitioin/FDDB-images/2002/08/11/big/img_591.jpg
1,1
180,41,359,282/159,77,345,263/2.164473072177929/0.6536946953903173

/content/drive/MyDrive/paper_face_recognitioin/FDDB-images/2002/08/26/big/img_265.jpg
3,3
60,20,150,154/66,56,155,146/1.8129051719805624/0.6071428571428571
60,20,150,154/294,80,402,187/0.9152948250613653/0.0
60,20,150,154/158,113,210,164/0.6254308857454176/0.0
156,87,211,171/66,56,155,146/1.8129051719805624/0.0
156,87,211,171/294,80,402,187/0.9152948250613653/0.0
156,87,211,171/158,113,210,164/0.6254308857454176/0.5789915966386555
296,47,385,187/66,56,155,146/1.8129051719805624/0.0
296,47,385,187/294,80,402,187/0.9152948250613653/0.6593406593406593
296,47,385,187/158,113,210,164/0.6254308857454176/0.0

/content/drive/MyDrive/paper_face_recognitioin/FDDB-images/2002/07/19/big/img_423.jpg
1,2
195,46,314,220/167,82,296,211/-0.0952408159904059/0.5381493506493507
195,46,314,220/417,2,453,38/-0.919680580013233/0.0

/content/drive/MyDriv

KeyboardInterrupt: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')